In [ ]:
import os
import io
import sys
import shutil
import pandas as pd
from megadetector.detection.run_detector_batch import load_and_run_detector_batch, write_results_to_file
from megadetector.utils import path_utils
from PIL import Image
import re
import json
from tqdm import tqdm

# Define paths
input_excel = '/home/tom-ratsakatika/VSCode/camera-traps/data/structured_data_with_deepfaune.xlsx'
processed_folder = '/media/tom-ratsakatika/CRUCIAL 4TB/FCC Camera Trap Data/processed/'
cropped_folder = '/media/tom-ratsakatika/CRUCIAL 4TB/FCC Camera Trap Data/cropped/'
empty_folder = os.path.join(processed_folder, 'Empty')
output_file = os.path.join(processed_folder, 'results.json')

os.makedirs(processed_folder, exist_ok=True)
os.makedirs(empty_folder, exist_ok=True)

data = pd.read_excel(input_excel)

data['New File Name'] = ''

def sanitize_label(label):
    return re.sub(r'[\\/*?:"<>|]', "", label)

file_counter = 1


In [ ]:
output_buffer = sys.stdout

sys.stdout = io.StringIO()

# Copy images to corresponding label folders with new names
for index, row in tqdm(data.iterrows(), total=data.shape[0], desc="Copying images"):
    original_path = row['Full File Name']
    label = sanitize_label(row['Ground_Truth'])
    new_name = f"{file_counter:06d}" + os.path.splitext(original_path)[1]  # Generate an incremental unique name
    new_folder = os.path.join(processed_folder, label)
    os.makedirs(new_folder, exist_ok=True)
    new_path = os.path.join(new_folder, new_name)
    
    try:
        shutil.copy2(original_path, new_path)
        data.at[index, 'New File Name'] = new_path
        file_counter += 1
    except FileNotFoundError:
        print(f"Error: {original_path} not found. Skipping.")

updated_excel = input_excel.replace('.xlsx', '_updated.xlsx')
data.to_excel(updated_excel, index=False)
print("Copying complete. Updated Excel saved at:", updated_excel)

In [ ]:
# Recursively find images in the processed folder
image_file_names = path_utils.find_images(processed_folder, recursive=True)

# confidence threshold for MegaDetector
confidence_threshold = 0.2

sys.stdout = io.StringIO()

# Run MegaDetector
results = load_and_run_detector_batch('MDV5A', image_file_names, confidence_threshold)

# Write results to output file
write_results_to_file(results, output_file, relative_path_base=processed_folder, detector_file='MDV5A')

In [ ]:
import json
import shutil
import os
from tqdm import tqdm
from PIL import Image
import sys

confidence_threshold = 0.2

with open(output_file) as f:
    detection_results = json.load(f)

for image_data in tqdm(detection_results['images'], desc="Cropping images"):
    image_path = processed_folder + image_data['file']
    detections = image_data['detections']
    
    if not detections:
        # Ensure source and destination are not the same
        dest_path = os.path.join(empty_folder, os.path.basename(image_path))
        if image_path != dest_path:
            shutil.copy2(image_path, empty_folder)
        # os.remove(image_path)
        continue
    
    cropped_counter = 1

    for i, detection in enumerate(detections):
        if detection['conf'] < confidence_threshold:
            continue
        bbox = detection['bbox']
        img = Image.open(image_path)
        width, height = img.size
        left = bbox[0] * width
        top = bbox[1] * height
        right = left + bbox[2] * width
        bottom = top + bbox[3] * height
        cropped_img = img.crop((left, top, right, bottom))

        # Create new file path lower extension and _# for number of detections
        dir_name, base_name = os.path.split(image_path)
        base, ext = os.path.splitext(base_name)
        new_file_name = f"{base}_{cropped_counter}{ext.lower()}"
        new_image_path = os.path.join(dir_name, new_file_name)
        
        cropped_img.save(new_image_path)

        cropped_counter += 1

sys.stdout = output_buffer


In [ ]:
def move_files_with_underscore(processed_folder, cropped_folder):
    # First, count the total number of files to be moved
    total_files = sum(len(files) for _, _, files in os.walk(processed_folder) if any("_" in file for file in files))
    
    with tqdm(total=total_files, desc="Moving files", unit="file") as pbar:
        for root, dirs, files in os.walk(processed_folder):
            for file in files:
                if "_" in file:
                    source_path = os.path.join(root, file)
                    relative_path = os.path.relpath(root, processed_folder)
                    dest_dir = os.path.join(cropped_folder, relative_path)
                    os.makedirs(dest_dir, exist_ok=True)
                    
                    dest_path = os.path.join(dest_dir, file)
                    
                    shutil.move(source_path, dest_path)
                    pbar.update(1)
                    print(f"Moved {source_path} to {dest_path}")


move_files_with_underscore(processed_folder, cropped_folder)


In [ ]:
def count_files_in_subfolders(folder):
    file_count = {}
    for root, dirs, files in os.walk(folder):
        num_files = len(files)
        file_count[root] = num_files
    return file_count

file_counts = count_files_in_subfolders(cropped_folder)

for folder, count in file_counts.items():
    print(f"{folder}: {count} files")

In [ ]:
import os
import shutil
import pandas as pd
from sklearn.model_selection import train_test_split
from collections import defaultdict

# Define paths
input_excel_updated = '/home/tom-ratsakatika/VSCode/camera-traps/data/structured_data_with_deepfaune_updated.xlsx'
output_base_folder = '/media/tom-ratsakatika/CRUCIAL 4TB/FCC Camera Trap Data/split_data'

train_folder = os.path.join(output_base_folder, 'train')
val_folder = os.path.join(output_base_folder, 'val')
test_folder = os.path.join(output_base_folder, 'test')

for folder in [train_folder, val_folder, test_folder]:
    os.makedirs(folder, exist_ok=True)

df = pd.read_excel(input_excel_updated)

classes_of_interest = {
    'badger', 'ibex', 'red deer', 'chamois', 'cat', 'goat', 'roe deer', 'dog', 'squirrel', 'equid', 'genet',
    'hedgehog', 'lagomorph', 'wolf', 'lynx', 'marmot', 'micromammal', 'mouflon', 'sheep', 'mustelid', 'bird', 
    'bear', 'nutria', 'fox', 'wild boar', 'cow'
}

df['class'] = df['New File Name'].apply(lambda x: os.path.basename(os.path.dirname(x)).lower())
df = df[df['class'].isin(classes_of_interest)]

sequence_dict = defaultdict(list)
for _, row in df.iterrows():
    original_file_name = os.path.basename(row['New File Name'])
    sequence_dict[row['seqnum']].append((row['class'], original_file_name))

sequence_ids = list(sequence_dict.keys())
train_ids, temp_ids = train_test_split(sequence_ids, test_size=0.3, random_state=42)
val_ids, test_ids = train_test_split(temp_ids, test_size=0.5, random_state=42)

def move_files(sequence_ids, target_folder):
    for seq_id in sequence_ids:
        for class_label, original_file_name in sequence_dict[seq_id]:
            cropped_files = [f for f in os.listdir(os.path.join(cropped_folder, class_label)) if f.startswith(original_file_name.split('.')[0])]
            for cropped_file in cropped_files:
                source_path = os.path.join(cropped_folder, class_label, cropped_file)
                target_dir = os.path.join(target_folder, class_label)
                os.makedirs(target_dir, exist_ok=True)
                shutil.move(source_path, target_dir)

move_files(train_ids, train_folder)
move_files(val_ids, val_folder)
move_files(test_ids, test_folder)

In [20]:
import os
import pandas as pd
from collections import defaultdict

def count_files_in_directory(directory):
    class_counts = defaultdict(int)
    for root, _, files in os.walk(directory):
        for file in files:
            class_label = os.path.basename(root)
            class_counts[class_label] += 1
    return class_counts


train_counts = count_files_in_directory(train_folder)
val_counts = count_files_in_directory(val_folder)
test_counts = count_files_in_directory(test_folder)

counts_df = pd.DataFrame({
    'Train': train_counts,
    'Validation': val_counts,
    'Test': test_counts
}).fillna(0).astype(int)

counts_df['Total'] = counts_df.sum(axis=1)

counts_df.loc['Total'] = counts_df.sum(axis=0)

counts_df['Train %'] = (counts_df['Train'] / counts_df['Total']) * 100
counts_df['Validation %'] = (counts_df['Validation'] / counts_df['Total']) * 100
counts_df['Test %'] = (counts_df['Test'] / counts_df['Total']) * 100

counts_df['Train %'] = counts_df['Train %'].round(0).astype(int)
counts_df['Validation %'] = counts_df['Validation %'].round(0).astype(int)
counts_df['Test %'] = counts_df['Test %'].round(0).astype(int)

counts_df = counts_df.sort_index()

cols = ['Total', 'Train', 'Validation', 'Test', 'Train %', 'Validation %', 'Test %']
counts_df = counts_df[cols]

print(counts_df)


           Total  Train  Validation  Test  Train %  Validation %  Test %
Total      24562  17336        3669  3557       71            15      14
badger       417    305          59    53       73            14      13
bear        3207   2329         352   526       73            11      16
bird         939    695         123   121       74            13      13
cat          356    276          25    55       78             7      15
chamois       41     34           4     3       83            10       7
cow          348    253          65    30       73            19       9
dog         1110    894         115   101       81            10       9
equid        446    214          50   182       48            11      41
fox         2286   1608         328   350       70            14      15
goat        1593   1350         165    78       85            10       5
lagomorph    107     81          15    11       76            14      10
lynx        1172    764         224   184       65 